## Searching for Collections in MAAP

These examples walk through the MAAP API functionality of searching for collections based on specific parameters. Collections are groupings of files that share the same product specification. Searching for collections can be useful for finding individual files, known as granules, which are used for processing.

We begin by importing the `MAAP` package and creating a new MAAP class.

In [6]:
# import the MAAP package to handle queries
from maap.maap import MAAP

# import printing package to help display outputs
from pprint import pprint

# invoke the MAAP search client
maap = MAAP()

We can use the `maap.searchCollection` function to return a list of desired collections. Before using this function, let's use the `help` function to view the specific arguments and keywords for `maap.searchCollection`.

In [7]:
# view help for the searchCollection function
help(maap.searchCollection)

Help on method searchCollection in module maap.maap:

searchCollection(limit=100, **kwargs) method of maap.maap.MAAP instance
    Search the CMR collections
    :param limit: limit of the number of results
    :param kwargs: search parameters
    :return: list of results (<Instance of Result>)



The help text is showing that `maap.searchCollection` accepts a limit and search parameters. The limit parameter limits the number of resulting collections returned by `maap.searchCollection`. Note that `limit=100` means that the *default limit* for results from the MAAP API is 100. `maap.searchCollection` accepts any additional search parameters that are included in the CMR. For a list of accepted parameters, please refer to the [CMR Search Collections API reference](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#collection-search-by-parameters).

In this example we will explore search options including:

1. Finding all Collections
2. Searching by temporal filter
3. Searching by spatial filter
4. Using the results from one search as inputs into another
5. Searching by additional attributes

### Finding all Collections

Here we will demonstrate how to create a list containing all of the collections contained within the CMR. To do this, we will use the `maap.searchCollection` function without any additional search parameters. 

In [4]:
# search all collections
results = maap.searchCollection(cmr_host="cmr.earthdata.nasa.gov")

# print the number of collections
pprint(f'Got {len(results)} results')

'Got 100 results'


We get 100 results because of the default page limit. The result from the MAAP API is a list of collections where each element in the list is the metadata for that particular collection. Note that as more collections are added to the CMR, it is important to remember that the default limit on results is 100. To change the limit, type `limit=` and then a value within the parentheses after `maap.searchCollection()`.

Let's look at the metadata for the first collection in our list of results (`results[0]`) using `pprint`. For formatting purposes, we can use the `depth` parameter to control the number of levels of metadata detail to display. By default, there is no constraint on the depth. By setting a `depth` parameter (in this case `depth=2`), we can ensure that the next contained level is replaced by an ellipsis.

In [8]:
# print the metadata for the first collection
# we use the depth parameter to set the layer of metadata detail in the results, with (1) having the least detail
# (1) displays the concept ID, format, and revision ID
# adjust the depth to a larger value (6) if you would like to view all of the metadata
pprint(results[0],depth=2)

{'Collection': {'ArchiveCenter': 'BCO-DMO',
                'Campaigns': {...},
                'CollectionDataType': 'SCIENCE_QUALITY',
                'CollectionState': 'PLANNED',
                'Contacts': {...},
                'DOI': {...},
                'DataFormat': 'ascii',
                'DataSetId': '"The Omnivores Dilemma": The Effect of Autumn '
                             'Diet on Winter Physiology and Condition of '
                             'Juvenile Antarctic Krill',
                'Description': 'Antarctic krill are essential in the Southern '
                               'Ocean as they support vast numbers of marine '
                               'mammals, seabirds and fishes, some of which '
                               'feed almost exclusively on krill. Antarctic '
                               'krill also constitute a target species for '
                               'industrial fisheries in the Southern Ocean. '
                               'T

The `Collection` key has all of the collection information including attributes, the archive center, spatial, and temporal information. The `concept-id` is a unique identifier for this collection. It can be used to further refine search results from the CMR, such as when searching for granule information.

### Searching by Temporal Filter

Here we use a temporal filter to narrow down our results using the `temporal` keyword in our search. The temporal keyword takes datetime information in a [specific format](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#c-temporal). The date format used is `YYYY-MM-DDThh:mm:ssZ` and temporal search criteria may be either a single date or a date range. If one date is provided then it can be inferred as the start or end date. To define a start date and return all collections after the date, put a comma after the date (`YYYY-MM-DDThh:mm:ssZ,`). To define a end date and return all granules prior to the data, put a comma before the date (`,YYYY-MM-DDThh:mm:ssZ`). Lastly, to get a date range, provide the start date and end date separated by a comma (`YYYY-MM-DDThh:mm:ssZ,YYYY-MM-DDThh:mm:ssZ`).

In this example we will search for one month of data.

In [11]:
datetimeRange = '2000-01-01T00:00:00Z,2000-01-31T23:59:59Z' # specify datetime range to search for data in January 2000

results = maap.searchCollection(
    cmr_host = "cmr.earthdata.nasa.gov",
    temporal = datetimeRange
)
pprint(f'Got {len(results)} results')

'Got 100 results'


In [12]:
collectionName = results[0]['Collection']['ShortName'] # get the collection short name
collectionDate = results[0]['Collection']['Temporal']['RangeDateTime']['BeginningDateTime'] # get the collection start time

pprint(
    f'Collection {collectionName} was acquired starting at {collectionDate}', width=100)


'Collection GLDAS_NOAH025_3H was acquired starting at 2000-01-01T00:00:00.000Z'


It appears the first result correctly matches with the beginning and ending temporal search parameters. Keep in mind that the results are limited to 100 so the final collection returned may not match the end date that was searched for.

### Searching by Spatial Filter

Here we will illustrate how to search for collections by a spatial filter. There are a couple of [spatial filters available to search by](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#c-spatial) in the CMR including point, line, polygon, and bounding box. In this example, we will explore filtering with a bounding box which is a sequence of four latitude and longitude values in the order of `[W,S,E,N]`. 

In [13]:
collectionDomain = '-42,10,42,20' # specify bounding box to search by

results = maap.searchCollection(
    cmr_host = "cmr.earthdata.nasa.gov",    
    bounding_box = collectionDomain
)
pprint(f'Got {len(results)} results')

'Got 100 results'


In [14]:
collectionName = results[3]['Collection']['ShortName'] # get a collection short name
collectionGeometry = results[3]['Collection']['Spatial']['HorizontalSpatialDomain']['Geometry'] # grab the spatial information from collection

pprint(f'Collection {collectionName} was acquired within the following geometry: ', width=100)
pprint(collectionGeometry)


'Collection gov.noaa.nodc:0000029 was acquired within the following geometry: '
{'BoundingRectangle': {'EastBoundingCoordinate': '-16.25',
                       'NorthBoundingCoordinate': '46.263167',
                       'SouthBoundingCoordinate': '0.766667',
                       'WestBoundingCoordinate': '-124.041667'},
 'CoordinateSystem': 'CARTESIAN'}


We can see from the first collection that the spatial coordinates of the collection intersect our search box.